In [197]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from rdkit import Chem
from rdkit.Chem import AllChem
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import PCA
import concurrent.futures
import csv

SyntaxError: unexpected EOF while parsing (<ipython-input-118-9caf311a3721>, line 1)

In [80]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [125]:
d_train = df_train.head()
d_test = df_test.head()
d_test['smiles'][d_test['smiles'] == 'c1sc(-c2cnc3c(c2)c2nsnc2c2cc4cccnc4cc32)c2cc[nH]c12'].index[0]

0

In [82]:
#store gap values
Y_train = d_train.gap.values
#delete 'Id' column
d_test = d_test.drop(['Id'], axis=1)
#delete 'gap' column
d_train = d_train.drop(['gap'], axis=1)

In [194]:
i = 0
def getBin(smile):
    global i
    i += 1
    print i
    mol = Chem.MolFromSmiles(smile)
    fprint = Chem.rdmolops.RDKFingerprint(mol)
    return list(map(int, fprint.ToBitString()))

In [195]:
with concurrent.futures.ProcessPoolExecutor() as executor:
    Bits = executor.map(getBin, df_test['smiles'].iloc[1:10], chunksize=10)
    
#     fPrintData_train = []
#     fPrintData_test = []
#     for smile in df_train.iloc[:len(df_train)/4]['smiles']:
#         mol = Chem.MolFromSmiles(smile)
#         fprint = Chem.rdmolops.RDKFingerprint(mol)
#         fPrintData.append(list(map(int, fprint.ToBitString())))

#     for smile in df_test['smiles']:
#         mol = Chem.MolFromSmiles(smile)
#         fprint = Chem.rdmolops.RDKFingerprint(mol)
#         fPrintData.append(list(map(int, fprint.ToBitString())))

1
1
1
1
2
3
2
4
2


In [198]:
with open("FingerPrintFeatures.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(Bits)

In [ ]:
fprintDF_train = DataFrame(fPrintData_train)
fprintDF_test = DataFrame(fPrintData_test)

In [ ]:
fprintDF_train.to_csv('fPrint_train.csv')
fprintDF_train.to_csv('fPrint_test.csv')
df_train['gap'].to_csv('gapValues.csv')

In [90]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)

In [87]:
NA_train = []
NA_test = []
GNHA_train = []
GNHA_test = []
NB_train = []
NB_test = []
NC_train = []
NC_test = []
GAA_train = []
GAA_test = []
non_Carbon_train = []
non_Carbon_test = []
Mol_mass_train = []
Mol_mass_test = []

for smile in df_train['smiles']:
    non_c_counter = 0
    mass = 0
    mol = Chem.MolFromSmiles(smile)
    NA_train.append(mol.GetNumAtoms())
    GNHA_train.append(mol.GetNumHeavyAtoms())
    NB_train.append(mol.GetNumBonds())
    NC_train.append(mol.GetNumConformers())
    GAA_train.append(len(mol.GetAromaticAtoms()))
    for i in range(mol.GetNumAtoms()):
        AN = mol.GetAtomWithIdx(i)
        if AN.GetAtomicNum() != 6:
            non_c_counter += 1
        mass += AN.GetMass()
    non_Carbon_train.append(non_c_counter)
    Mol_mass_train.append(mass)
    
    
    
for smile in df_test['smiles']:
    non_c_counter = 0
    mass = 0
    mol = Chem.MolFromSmiles(smile)
    NA_test.append(mol.GetNumAtoms())
    GNHA_test.append(mol.GetNumHeavyAtoms())
    NB_test.append(mol.GetNumBonds())
    NC_test.append(mol.GetNumConformers())
    GAA_test.append(len(mol.GetAromaticAtoms()))
    for i in range(mol.GetNumAtoms()):
        AN = mol.GetAtomWithIdx(i)
        if AN.GetAtomicNum() != 6:
            non_c_counter += 1
        mass += AN.GetMass()
    non_Carbon_test.append(non_c_counter)
    Mol_mass_test.append(mass)

0

In [26]:
# df_Simple_Train =
# df_Simple_Test = 

In [84]:
d_train['NumAtms'] = NA_train
d_train['HvyAtms'] = GNHA_train
d_train['NumBnds'] = NB_train
d_test['NumAtms'] = NA_test
d_test['HvyAtms'] = GNHA_test
d_test['NumBnds'] = NB_test
d_train['Conformers'] = NC_train
d_train['AromAtms'] = GAA_train
d_test['Conformers'] = NC_test
d_test['AromAtms'] = GAA_test
d_train['Mass'] = Mol_mass_train
d_train['Non-Carbon'] = non_Carbon_train
d_test['Mass'] = Mol_mass_test
d_test['Non-Carbon'] = non_Carbon_test

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_254,feat_255,feat_256,NumAtms,HvyAtms,NumBnds,Conformers,AromAtms,Mass,Non-Carbon
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,27,27,32,0,27,461.390,8
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,25,25,30,0,14,336.417,4
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,27,27,33,0,23,386.472,6
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,27,27,33,0,20,362.431,4
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,29,29,35,0,29,388.327,9


In [97]:
data_train = pd.DataFrame(data=[NA_train, GNHA_train, NB_train, NC_train, GAA_train, Mol_mass_train, non_Carbon_train]).transpose()
data_train.columns = ['NumAtms', 'HvyAtms', 'NumBnds', 'Conformers', 'AromAtms', 'Mass', 'Non-Carbon']
data_test = pd.DataFrame(data=[NA_test, GNHA_test, NB_test, NC_test, GAA_test, Mol_mass_test, non_Carbon_test]).transpose()
data_test.columns = ['NumAtms', 'HvyAtms', 'NumBnds', 'Conformers', 'AromAtms', 'Mass', 'Non-Carbon']

In [98]:
data_train.to_csv('PurelyEngineered_train.csv')
data_test.to_csv('PurelyEngineered_test.csv')

In [33]:
train_smiles = train_x['smiles']
test_smiles = test_x['smiles']

In [36]:
df_train = df_train.drop(['smiles'], axis=1)
df_test = df_test.drop(['smiles'], axis=1)

In [91]:
x_train, x_test, y_train, y_test = train_test_split(data, Y_train, test_size = .33)

In [92]:
Lasso = LassoCV()
Lasso.fit(x_train, y_train)
Lasso_pred = Lasso.predict(x_test)

Lasso_error = mean_squared_error(y_test, Lasso_pred)

In [93]:
Lasso_error

0.081406728282691354

In [79]:
Ridge = RidgeCV()
Ridge.fit(x_train, y_train)
Ridge_pred = Ridge.predict(x_test)

Ridge_error = mean_squared_error(y_test, Ridge_pred)

In [80]:
l1_rtio = Lasso_error / (Lasso_error + Ridge_error)

EN = ElasticNetCV(l1_ratio = [l1_rtio, .1, .9, .95, .99, 1])
EN.fit(x_train, y_train)
EN_pred = EN.predict(x_test)

EN_error = mean_squared_error(y_test, EN_pred)

In [ ]:
Ada = AdaBoostRegressor(DecisionTreeRegressor(), learning_rate=0.05)
params = {'base_estimator__max_depth':list(range(1,6))}
ada_cv = GridSearchCV(Ada, params, cv = 5)
ada_cv.fit(x_train, y_train)

In [57]:
# print Lasso_error
# print Ridge_error
# print EN_error
print cross_val_score(LassoCV(), df_train, Y_train)
print cross_val_score(RidgeCV(), df_train, Y_train)
print cross_val_score(ElasticNetCV(l1_ratio = l1_rtio), df_train, Y_train)

[ 0.5429816   0.54284339  0.54041939]
[ 0.54604865  0.54571122  0.54324111]
[ 0.54253733  0.54239732  0.54003766]


In [59]:
cross_val_score(LinearRegression(), df_train, Y_train)

array([ 0.54604906,  0.54571092,  0.54323894])

In [82]:
LR = LinearRegression()
LR.fit(x_train, y_train)
LR_pred = LR.predict(x_test)

LR_error = mean_squared_error(y_test, LR_pred)

In [83]:
print math.sqrt(LR_error)
print math.sqrt(Ridge_error)
print math.sqrt(Lasso_error)
print math.sqrt(EN_error)
print math.sqrt(Ada_error)

0.274973640215
0.274973000858
0.275906454477
0.275906454477
0.313597671048


In [94]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
#smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
#df_all['smiles_len'] = pd.DataFrame(smiles_len)


'\nExample Feature Engineering\n\nthis calculates the length of each smile string and adds a feature column with those lengths\nNote: this is NOT a good feature and will result in a lower score!\n'

In [95]:
#Drop the 'smiles' column
df_all = df_all.drop(['smiles'], axis=1)
vals = df_all.values
X_train = vals[:test_idx]
X_test = vals[test_idx:]
print "Train features:", X_train.shape
print "Train gap:", Y_train.shape
print "Test features:", X_test.shape

Train features: (1000000, 256)
Train gap: (1000000,)
Test features: (824230, 256)


In [100]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
data_all = pd.concat((data_train, data_test), axis=0)
data_all.head()

#Drop the 'smiles' column
# data_all = data_all.drop(['smiles'], axis=1)
vals = data_all.values
X_train = vals[:test_idx]
X_test = vals[test_idx:]
print "Train features:", X_train.shape
print "Train gap:", Y_train.shape
print "Test features:", X_test.shape

Train features: (1000000, 7)
Train gap: (1000000,)
Test features: (824230, 7)


In [10]:
LR = LinearRegression()
LR.fit(X_train, Y_train)
LR_pred = LR.predict(X_test)

In [11]:
RF = RandomForestRegressor()
RF.fit(X_train, Y_train)
RF_pred = RF.predict(X_test)

In [101]:
Lasso = LassoCV()
Lasso.fit(X_train, Y_train)
Lasso_pred = Lasso.predict(X_test)

In [106]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [107]:
write_to_file("EngineeredLassoPreds.csv", Lasso_pred)

In [ ]:
write_to_file("sample1.csv", LR_pred)
write_to_file("sample2.csv", RF_pred)

In [13]:
print LR_pred
print RF_pred

[ 1.61237192  1.65141153  1.53785944 ...,  1.61237192  1.77372119
  1.7494111 ]
[ 1.46531041  1.65460187  1.54674697 ...,  1.46531041  1.50772525
  1.82709096]


In [14]:
x_train, x_test, y_train, y_test = train_test_split(df_train, Y_train, test_size = 1000, train_size = 1000)

In [ ]:
Var90 = False
i = 0

# Run PCA with increasing number of components until 90% variance
while not Var90:
    i += 1
    pca = PCA(n_components = i)
    pca.fit(x_train, y_train)
    if np.sum(pca.explained_variance_ratio_) >= .9:
        Var90 = True

print i
        
# Fit the pca model to the train and test data
X_PCA = pca.transform(x_train)
X_PCA_Test = pca.transform(x_test)

In [ ]:
LR_pca = LinearRegression()
LR_reg = LinearRegressionRegressionrRegression()
LR_pca.fit(X_PCA, t_train)
LR_reg.fit(x_train, y_train)
LR_pca_pred = LR_pca.predict(X_PCA_Test)
LR_reg_pred = LR_reg.prmean_squared_errort(x_test)

print mean_squared_error(y_test, LR_pca_pred)
print mean_squared_error(y_test, LR_reg_pred)